In [ ]:
import torch
from torch.nn import Module
import torch.nn.functional as F
import brevitas.nn as qnn
import torch 
import numpy as np
import torch.nn as nn
import brevitas.onnx as bo

input_dim = 72
output_dim = 11

x_data = np.load('inputs.npy', fix_imports=False)
y_data = np.load('labels.npy', fix_imports=False)

In [ ]:
#hardtanh version
class MLP_quantized_exportable(Module):
    def __init__(self):
        super(MLP_quantized_exportable, self).__init__()
        self.input   = qnn.QuantLinear(input_dim, 256, bias=False, 
                                     weight_quant_type=QuantType.INT, 
                                     weight_bit_width=8)
        self.input_relu = qnn.QuantHardTanh(quant_type=QuantType.INT, bit_width=8)
        
        self.h1   = qnn.QuantLinear(256, 256, bias=False, 
                                     weight_quant_type=QuantType.INT, 
                                     weight_bit_width=8)
        self.relu1 = qnn.QuantHardTanh(quant_type=QuantType.INT, bit_width=8)
        
        self.h2   = qnn.QuantLinear(256, 128, bias=False, 
                                     weight_quant_type=QuantType.INT, 
                                     weight_bit_width=8)
        self.relu2 = qnn.QuantHardTanh(quant_type=QuantType.INT, bit_width=8)
        
        self.h3   = qnn.QuantLinear(128, output_dim, bias=False, 
                                     weight_quant_type=QuantType.INT, 
                                     weight_bit_width=8)

        self.output   = qnn.QuantSigmoid(quant_type=QuantType.FP, bit_width=8)

    def forward(self, x):
        x = self.input(x)
        x = self.input_relu(x)
        
        x = self.h1(x)
        x = self.relu1(x)
        
        x = self.h2(x)
        x = self.relu2(x)
        
        x = self.h3(x)
        return self.output(x)

In [ ]:
bo.export_finn_onnx(model, (input_dim), "fpga_model.onnx")